In [ ]:
# blink detection
# angle detection
# iris tracking
# 2020 rule
# distance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# for n,i in enumerate(points):
#     if n==31:
#         cv2.putText(frame, str(n) , (i[0],i[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 0), 2, cv2.LINE_AA)#3 size h 2 thickness
# plt.imshow(frame)

In [ ]:
#blepblink 
import cv2, numpy as np, glob, os,shutil, keras, matplotlib.pyplot as plt,tensorflow as tf,matplotlib.mlab as mplt, sys
from PIL import Image
from matplotlib import style
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout,ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split as tt
import scipy.io.wavfile
from math import sqrt
import librosa
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve,f1_score,average_precision_score,confusion_matrix
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler as sd
from sklearn.model_selection import StratifiedKFold
import subprocess
from sklearn.decomposition import PCA
from IPython.display import Audio
from keras.models import Model
from keras import optimizers
from imutils import face_utils
import dlib,math


detector= dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./drive/My Drive/Copy of shape_predictor_68_face_landmarks.dat')

Using TensorFlow backend.


In [ ]:
drop=0.8
learning_rate=.0001
epoch=50
count=0

model = Sequential()
model.add(Convolution2D(32, (2,2), input_shape=(50,50,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Convolution2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


model.add(Convolution2D(32, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Convolution2D(64, 2, activation='relu',padding='same'))
#model.add(ZeroPadding2D((1, 1)))
model.add(MaxPooling2D(pool_size=2,padding='same'))
#model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.30))
#   model.add(Dense(512, activation='relu'))
#   model.add(Dense(1024, activation='relu'))
model.add(Dropout(drop))
model.add(Dense(2))
model.add(Activation('softmax'))
model.load_weights("./drive/My Drive/colour.hdf5")
adam=optimizers.Adam(lr=learning_rate)
model.compile(optimizer=adam,loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def point(gray):
    shape1=np.array([])
    rects1=detector(gray,0)
    for rect in rects1:
        shape = predictor(gray,rect)
        shape1 = face_utils.shape_to_np(shape)
    if(shape1.any()==0):
        return []
    else:
        return shape1

In [ ]:
def eyes(gray,shape1):
    fe1=shape1[45][0] - shape1[42][0]#eye lft ke liye
    x1=shape1[42][0]
    x1=x1-2
    x2= shape1[45][0] 
    x2=x2+2
    y1=shape1[43][1]
    y1=y1-18
    y2=shape1[46][1]
    y2=y2+12
    if(x1<0):
        x1=0
    if(y1<0):
        y1=0
    if(x2>frame.shape[1]):
        x2=frame.shape[1]
    if(y2>frame.shape[0]):
        y2=frame.shape[0]   

    x_1=gray[y1:y2, x1:x2]

    fe2=shape1[39][0] - shape1[36][0]# fe2 eye 2 ke liye
    x1=shape1[36][0]
    x1=x1-2
    x2= shape1[39][0] 
    x2=x2+2
    y1=shape1[37][1]
    y1=y1-18
    y2=shape1[40][1]
    y2=y2+12
    if(x1<0):
        x1=0
    if(y1<0):
        y1=0
    if(x2>frame.shape[1]):
        x2=frame.shape[1]
    if(y2>frame.shape[0]):
        y2=frame.shape[0] 
    x_2=gray[y1:y2, x1:x2]

    return x_1,x_2

In [ ]:
def distance_init(D,W,points):
    P=abs(points[35][0]-points[31][0])
    return (P*D)/W

In [ ]:
#angle
def angle(frame,points,v=0):
    size=frame.shape
    image_points = np.array([(points[30][0], points[30][1]),(points[8][0], points[8][1]),
                            (points[36][0], points[36][1]),(points[45][0], points[45][1]),
                            (points[48][0], points[48][1]),(points[54][0], points[54][1])], dtype="double")#nose tip,chin,lip left right,eye left's left, right's right
    model_points = np.array([(0.0, 0.0, 0.0),(0.0, -330.0, -65.0),(-225.0, 170.0, -135.0),(225.0, 170.0, -135.0),(-150.0, -150.0, -125.0),(150.0, -150.0, -125.0)])
    # Nose tip, Chin , eye left corner ,Right eye right corner, Left Mouth corner ,Right mouth corner
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]],[0, focal_length, center[1]],[0, 0, 1]], dtype = "double")
    #print ("Camera Matrix :",camera_matrix)
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.cv2.SOLVEPNP_ITERATIVE)
    if v==1:
        (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
        for p in image_points:
            cv2.circle(frame, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
        p1 = ( int(image_points[0][0]), int(image_points[0][1]))
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
        cv2.line(frame, p1, p2, (255,0,0), 2)
        return frame
    else:
        one=rotation_vector[0][0]*180/3.14
        sec=rotation_vector[1][0]*180/3.14
        thd= rotation_vector[2][0]*180/3.14
        return one,sec,thd

In [ ]:
!wget https://images-na.ssl-images-amazon.com/images/I/51svmvAZdNL._SX355_.jpg

--2020-06-08 20:14:26--  https://images-na.ssl-images-amazon.com/images/I/51svmvAZdNL._SX355_.jpg
Resolving images-na.ssl-images-amazon.com (images-na.ssl-images-amazon.com)... 151.101.1.16, 151.101.65.16, 151.101.129.16, ...
Connecting to images-na.ssl-images-amazon.com (images-na.ssl-images-amazon.com)|151.101.1.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23748 (23K) [image/jpeg]
Saving to: ‘51svmvAZdNL._SX355_.jpg’

51svmvAZdNL._SX355_ 100%[===================>]  23.19K  --.-KB/s    in 0.01s   

2020-06-08 20:14:26 (1.82 MB/s) - ‘51svmvAZdNL._SX355_.jpg’ saved [23748/23748]



In [ ]:
!wget https://images.indianexpress.com/2018/07/delhi-boy-fb.jpg

--2020-06-08 20:05:35--  https://images.indianexpress.com/2018/07/delhi-boy-fb.jpg
Resolving images.indianexpress.com (images.indianexpress.com)... 23.5.10.125
Connecting to images.indianexpress.com (images.indianexpress.com)|23.5.10.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64937 (63K) [image/jpeg]
Saving to: ‘delhi-boy-fb.jpg’

delhi-boy-fb.jpg    100%[===================>]  63.42K  --.-KB/s    in 0.1s    

2020-06-08 20:05:36 (617 KB/s) - ‘delhi-boy-fb.jpg’ saved [64937/64937]



In [ ]:
#main
points=0
frame=cv2.imread("delhi-boy-fb.jpg")#51svmvAZdNL._SX355_.jpg")
gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
points=point(gray)
if len(points)==0:
    print("0")
else:
   le,re = eyes(gray,points)
   # left right eye

In [ ]:
# distance
#F=PD/W
#D=WF/P
# W actual width of object, we place it D distance apart, P pixel width of our object, F focal length of our camera. camera ki seedh me ho
d,w=input("enter the your distance and nose width in centimeters")
F=distance_init(d,w,points)
P=abs(points[35][0]-points[31][0])
D=(W*F)/P
print("Distance is ",D,"cms")

In [ ]:
#iris movement https://github.com/antoinelame/GazeTracking

In [ ]:
!git clone https://github.com/antoinelame/GazeTracking.git

Cloning into 'GazeTracking'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.


In [ ]:
os.listdir("GazeTracking")

In [ ]:
from GazeTracking.gaze_tracking.gaze_tracking import GazeTracking

In [ ]:
gaze = GazeTracking()

In [ ]:
gaze.refresh(frame)
gaze.pupil_left_coords(),gaze.pupil_right_coords()

(345, 343)

In [ ]:
angle(frame,points)

(-183.45465526913463, 16.110813390269513, -20.169831920909612)